In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv("C:/Users/lauta/Desktop/Internship DealRoom/data_scientist_intern_revenue_model.csv")

In [3]:
df.head(10)

,NAME,Revenue_2015,Revenue_2016,Revenue_2017,Revenue_2018,Revenue_2019,Employees_2017,Employees_2018,Employees_2019,Employees_2020,Revenue_2020(To predict)
0,LAKI,"19,039,781,738","24,671,631,399","24,573,762,847","22,608,855,366","26,574,540,757",NaN,805.0,790.0,785.0,NaN
1,YETS,"3,687,564,000","3,663,588,000","3,420,430,000","3,265,963,000","3,465,058,000",NaN,2394.0,2494.0,2548.0,NaN
2,PNMD,"3,139,300,000","3,214,000,000","3,196,900,000","3,193,700,000","3,166,000,000",6411.0,6506.0,7750.0,NaN,NaN
3,VKNF,"1,994,112,000","2,023,348,000","1,982,028,000","2,007,387,000","2,009,031,000",NaN,1925.0,1995.0,2050.0,NaN
4,HHVD,"761,248,939","1,121,284,088","1,245,810,056","1,317,864,838","1,779,401,992",NaN,5159.0,5599.0,5483.0,NaN
5,OWRH,"1,542,346,719","1,432,739,017","1,684,764,846","1,878,176,166","1,756,472,782",NaN,319.0,343.0,335.0,NaN
6,DQVE,"1,198,734,028","1,377,446,067","1,461,826,444","1,657,818,945","1,663,037,290",NaN,3405.0,3377.0,3096.0,NaN
7,SMBV,"1,423,357,300","1,455,164,329","1,406,572,220","1,534,895,751","1,667,789,302",197.0,203.0,209.0,NaN,NaN
8,LLQC,"1,462,664,130","1,343,274,474","1,487,356,029","1,562,650,199","1,575,028,199",3492.0,3748.0,3845.0,NaN,NaN
9,XXEK,"1,592,032,000","1,585,729,000","1,476,923,000","1,451,335,000","1,513,090,000",389.0,425.0,442.0,NaN,NaN


In [4]:
#Checking general info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17424 entries, 0 to 17423
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   NAME                      17424 non-null  object 
 1   Revenue_2015              10483 non-null  object 
 2   Revenue_2016              11737 non-null  object 
 3   Revenue_2017              12531 non-null  object 
 4   Revenue_2018              13576 non-null  object 
 5   Revenue_2019              10245 non-null  object 
 6   Employees_2017            2180 non-null   float64
 7   Employees_2018            5844 non-null   float64
 8   Employees_2019            13135 non-null  float64
 9   Employees_2020            9725 non-null   float64
 10  Revenue_2020(To predict)  0 non-null      float64
dtypes: float64(5), object(6)
memory usage: 1.5+ MB


#### Data Cleaning

In [5]:
#Dropping unncessary column
df.pop('Revenue_2020(To predict)')

#Correcting the names of the columns
df.rename(columns = {'Revenue_2015 ': 'Revenue_2015'}, inplace=True)

#Correcting the format of missing information in the dataset
df.replace(' -   ', np.nan, inplace=True)

#Taking out the ',' to convert into numeric columns
df["Revenue_2015"] = df["Revenue_2015"].str.replace(',', '')
df["Revenue_2016"] = df["Revenue_2016"].str.replace(',', '')
df["Revenue_2017"] = df["Revenue_2017"].str.replace(',', '')
df["Revenue_2018"] = df["Revenue_2018"].str.replace(',', '')
df["Revenue_2019"] = df["Revenue_2019"].str.replace(',', '')

#Converting revenue columns into numeric to proccess them
df["Revenue_2015"] = pd.to_numeric(df["Revenue_2015"])
df["Revenue_2016"] = pd.to_numeric(df["Revenue_2016"])
df["Revenue_2017"] = pd.to_numeric(df["Revenue_2017"])
df["Revenue_2018"] = pd.to_numeric(df["Revenue_2018"])
df["Revenue_2019"] = pd.to_numeric(df["Revenue_2019"])

#I noticed some data appeared as '1' when in fact it seems that it was supposed to be a NA value. Here I corrected this.
df['Revenue_2015'].values[df['Revenue_2015'] == 1] = np.nan
df['Revenue_2016'].values[df['Revenue_2016'] == 1] = np.nan
df['Revenue_2017'].values[df['Revenue_2017'] == 1] = np.nan
df['Revenue_2018'].values[df['Revenue_2018'] == 1] = np.nan
df['Revenue_2019'].values[df['Revenue_2019'] == 1] = np.nan

## Exploratory Analysis

In [6]:
df.corr()

,Revenue_2015,Revenue_2016,Revenue_2017,Revenue_2018,Revenue_2019,Employees_2017,Employees_2018,Employees_2019,Employees_2020
Revenue_2015,1.000000,0.975891,0.970413,0.948331,0.956856,0.486929,0.078318,0.084387,0.070315
Revenue_2016,0.975891,1.000000,0.991962,0.962105,0.961430,0.503408,0.101678,0.109150,0.093010
Revenue_2017,0.970413,0.991962,1.000000,0.983793,0.978679,0.544048,0.109343,0.117551,0.100618
Revenue_2018,0.948331,0.962105,0.983793,1.000000,0.993628,0.514907,0.114797,0.123631,0.108384
Revenue_2019,0.956856,0.961430,0.978679,0.993628,1.000000,0.528004,0.138917,0.150134,0.128557
Employees_2017,0.486929,0.503408,0.544048,0.514907,0.528004,1.000000,0.994442,0.986724,0.966659
Employees_2018,0.078318,0.101678,0.109343,0.114797,0.138917,0.994442,1.000000,0.999710,0.999186
Employees_2019,0.084387,0.109150,0.117551,0.123631,0.150134,0.986724,0.999710,1.000000,0.999786
Employees_2020,0.070315,0.093010,0.100618,0.108384,0.128557,0.966659,0.999186,0.999786,1.000000


We can see high correlatios between Revenues from each years and other years, 
and between quantities of employees from each year to other years but not so much in between this two types of variables
We can also see that the closer the year is (e.g. 2018 is closer to 2019 than 2016) the higher the correlation between variables

Nevertheless, the amount of missing data seems to be a challenge

In [7]:
#Checking percetanges of missing data per column
round(df.isnull().mean(),2)

NAME              0.00
Revenue_2015      0.41
Revenue_2016      0.34
Revenue_2017      0.29
Revenue_2018      0.23
Revenue_2019      0.42
Employees_2017    0.87
Employees_2018    0.66
Employees_2019    0.25
Employees_2020    0.44
dtype: float64

In [8]:
#Getting the Quantity of observations who are missing all data about past revenues
print("Quatity of Companies that we don't have any data about past revenues: " + str(len(df[df.iloc[:,1:6].isnull().sum(1)==5])))

Quatity of Companies that we don't have any data about past revenues: 177


#### Predictions on this companies' revenue would be very inaccurate given that the only data that we have is Quantity of Employees from past years and this not enough. But I will proceed to include their predictions given the nature of this excercise.

## Dealing with Missing Data

There are multiple ways to deal with missing data. In this case, given that there's a high linear correlation between the same types of variables from different years, I decided to use one of the most robust methods, which is the Multiple Imputation. However, the imputations will be done only on the basis of the high related variables (i.e. the revenue of a missing year will be imputed by the predictions of the revenue of others years but not on the quantity of employees).

Although this method if very robust, because of lack of data a lot of imputations will be just the mean of the variable. Furthermore, this will create an overstimation of the linear correlation of the data.

In [9]:
revenues = ['Revenue_2015','Revenue_2016','Revenue_2017','Revenue_2018','Revenue_2019']
employees = ['Employees_2017','Employees_2018','Employees_2019', 'Employees_2020']

df_revenues = df[revenues]
df_employees = df[employees]

In [10]:
lr = LinearRegression()
imputer = IterativeImputer(estimator = lr, max_iter=30)

df[revenues] = imputer.fit_transform(df_revenues)

imputer = IterativeImputer(estimator = lr, max_iter=30, min_value=0)


df[employees] = imputer.fit_transform(df_employees)

## Training Model

#### For the predicting task, I decided to use the last 4 years of revenues of each company and the current quantity of employees as features. First, I made a dataset for the year 2019 to use as training set and one for 2020 to use as the testset.

In [11]:
df_train = pd.DataFrame({'-4 Year' : df.Revenue_2015, '-3 Year': df.Revenue_2016, '-2 Year': df.Revenue_2017, '-1 Year': df.Revenue_2018, 'Employees': df.Employees_2019, 'Target': df.Revenue_2019})
test = pd.DataFrame({'-4 Year' : df.Revenue_2016, '-3 Year': df.Revenue_2017, '-2 Year': df.Revenue_2018, '-1 Year': df.Revenue_2019, 'Employees': df.Employees_2020})

In [12]:
#Splittin the training set into train and validation sets
train, validation = train_test_split(df_train, 
                                     train_size = 0.8, 
                                     test_size = 0.2,
                                    random_state=7)

y_train = train.pop('Target')
X_train = train

y_validation = validation.pop('Target')
X_validation = validation

In [13]:
#Getting the scores from cross validation of the training set
folds = KFold(n_splits = 5, shuffle = True, random_state = 7)
r2_scores = cross_val_score(lr, X_train, y_train, scoring='r2', cv=folds)
mae_scores = cross_val_score(lr, X_train, y_train, scoring='neg_mean_absolute_error', cv=folds)

print('Cross Validation Mean R2 Score for Training set with 5 Splits: ', round(np.mean(r2_scores),2))
print('Cross Validation Mean MAE Score for Training set with 5 Splits: ', round(np.mean(abs(mae_scores)),2))

Cross Validation Mean R2 Score for Training set with 5 Splits:  0.97
Cross Validation Mean MAE Score for Training set with 5 Splits:  3296073.56


#### The Mean of the 5 Fold Cross R2 Score Validation with a linear regression model is very high, meaning that the total variance of the target variable is almost completey explained by the dependent variables. The difference that a complex model (like neural networks) could bring to the accuracy would not be worth the explainability that this model has. For this reason, I decided to use the linear regression as the final model.

In [14]:
#Definite score of the validation set
model = lr.fit(X_train, y_train)
y_pred = model.predict(X_validation)
r2 = sklearn.metrics.r2_score(y_validation, y_pred)
mse = mean_absolute_error(y_validation, y_pred)
print('R2 Score is: ' , round(r2,2))
print('Mean Squared Error: ', round(mse,2))

R2 Score is:  0.99
Mean Squared Error:  2651602.96


#### Here we can see that the predictions from 2019 with this model miss the actual value on average by 2651602.96. We should expect that prediction for 2020 will have a similar or bigger error in each year.

In [15]:
#Making the predictions on the testset (Revenue of 2020)
df['Revenue_2020'] = model.predict(test)
df.to_csv("C:/Users/lauta/Desktop/Internship DealRoom/revenue_model_complete.csv", index=False)